# Kaggle
## Competição DSA de Machine Learning - Dezembro 2019

Versão 1.0.0: LB = 0.48866 CV = 0.463102
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado através do Auto_ViML

Versão 1.0.1: LB = 0.48991 CV = 0.462946
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado através do Auto_ViML (com novas features)

Versão 1.0.2: LB = 0.48915 CV = 0.464442
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado através do Auto_ViML (com agrupamento pela coluna v2)

Versão 1.0.3: LB = 0.88299 CV = 0.344659
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado interno

Versão 1.0.4: LB = 0.6041 CV = 0.461724
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado através do Auto_ViML (modificado v1)

Versão 1.0.5: LB = 0.48897 CV = 0.466055 (kfold)   CV = 0.466422 (StratifiedKFold)
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado com pacotes estatisticos

Versão 1.0.6: LB = 0.49521 CV = 0.466138 (kfold)
- modelo: LightGBM (com algumas otimizações)
- features engineering: Kernel_Feature_Engineering_v.1.0.0
- utilizado features categoricas no lgb

Versão 1.0.7: LB = ???? CV = 0.473055 (kfold)
- modelo: LightGBM (com algumas otimizações)
- features engineering: Kernel_Feature_Engineering_v.1.0.0
- aplicado scala nos dados numericos
- utilizado features categoricas no lgb

# Importando as bibliotecas

In [ ]:
# Importar os principais pacotes
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

import time
import datetime
import gc

# Evitar que aparece os warnings
import warnings
warnings.filterwarnings("ignore")

# Seta algumas opções no Jupyter para exibição dos datasets
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Variavel para controlar o treinamento no Kaggle
TRAIN_OFFLINE = True

In [ ]:
# Importa os pacotes de algoritmos
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier

# Importa pacotes do sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder


In [ ]:
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

# Carregando os dados de treino e teste

In [ ]:
def read_data():
    
    print('Carregando arquivo dataset_treino_new.csv....')
    train = pd.read_csv('../dataset/dataset_treino_new.csv')
    print('dataset_treino.csv tem {} linhas and {} colunas'.format(train.shape[0], train.shape[1]))

    print('Carregando arquivo dataset_teste_new.csv....')
    test = pd.read_csv('../dataset/dataset_teste_new.csv')
    print('dataset_teste.csv tem {} linhas and {} colunas'.format(test.shape[0], test.shape[1]))
    
    return train, test

In [ ]:
# Leitura dos dados
train, test = read_data()
df = train.append(test)
df = df.drop(columns = ['1'], axis = 1)

In [ ]:
# Verificar a quantidade de features numericas e categoricas

numerical_feats = df.dtypes[df.dtypes != "object"]
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = df.dtypes[df.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))

In [ ]:
df.head()

In [ ]:
for c in df.columns:
    col_type = df[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        df[c] =df[c].astype('category')

In [ ]:
scaler = StandardScaler()
for c in df.columns:
    col_type = df[c].dtype
    if col_type == 'float64' and c != 'target':
        df[c] = scaler.fit_transform(df[c].values.reshape(-1, 1))

In [ ]:
df.head()

# Algoritmo LigthGBM

In [ ]:
# Configurações Gerais

GENERATE_SUBMISSION_FILES = True
SUBMISSION_SUFIX = "_lgbm_v.1.0.7"
STRATIFIED_KFOLD = False
RANDOM_SEED = 737851
NUM_THREADS = 4
NUM_FOLDS = 10
EARLY_STOPPING = 1000

LIGHTGBM_PARAMS = {
    'boosting_type': 'goss',#'gbdt',
    'n_estimators': 10000,
    'learning_rate': 0.005134,
    'num_leaves': 54,
    'max_depth': 10,
    'subsample_for_bin': 240000,
    'reg_alpha': 0.436193,
    'reg_lambda': 0.479169,
    'colsample_bytree': 0.508716,
    'min_split_gain': 0.024766,
    'subsample': 1,
    'is_unbalance': False,
    'silent':-1,
    'verbose':-1
}

In [ ]:
# ------------------------- LIGHTGBM MODEL -------------------------

def run_model(data, categorical_feature = None):
    df = data[data['target'].notnull()]
    test = data[data['target'].isnull()]
    del_features = ['target']
    predictors = list(filter(lambda v: v not in del_features, df.columns))
    
    print("Train/valid shape: {}, test shape: {}".format(df.shape, test.shape))

    if not STRATIFIED_KFOLD:
        folds = KFold(n_splits= NUM_FOLDS, shuffle=True, random_state= RANDOM_SEED)
    else:
        folds = StratifiedKFold(n_splits= NUM_FOLDS, shuffle=True, random_state= RANDOM_SEED)

    # Hold oof predictions, test predictions, feature importance and training/valid auc
    oof_preds = np.zeros(df.shape[0])
    sub_preds = np.zeros(test.shape[0])
    importance_df = pd.DataFrame()
    eval_results = dict()
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(df[predictors], df['target'])):
        train_x, train_y = df[predictors].iloc[train_idx], df['target'].iloc[train_idx]
        valid_x, valid_y = df[predictors].iloc[valid_idx], df['target'].iloc[valid_idx]

        params = {'random_state': RANDOM_SEED, 'nthread': NUM_THREADS}
        
        clf = LGBMClassifier(**{**params, **LIGHTGBM_PARAMS})
        
        if not categorical_feature.empty:
            clf.fit(train_x, train_y, 
                    eval_set=[(train_x, train_y), (valid_x, valid_y)],
                    eval_metric='logloss', 
                    verbose=400, 
                    early_stopping_rounds= EARLY_STOPPING)
        else:
            clf.fit(train_x, train_y, 
                    eval_set=[(train_x, train_y), (valid_x, valid_y)],
                    eval_metric='logloss', 
                    verbose=400, 
                    early_stopping_rounds=EARLY_STOPPING,
                    feature_name= list(df[predictors].columns), 
                    categorical_feature= categorical_feature)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test[predictors], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        # Feature importance by GAIN and SPLIT
        fold_importance = pd.DataFrame()
        fold_importance["feature"] = predictors
        fold_importance["gain"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance["split"] = clf.booster_.feature_importance(importance_type='split')
        importance_df = pd.concat([importance_df, fold_importance], axis=0)
        eval_results['train_{}'.format(n_fold+1)]  = clf.evals_result_['training']['binary_logloss']
        eval_results['valid_{}'.format(n_fold+1)] = clf.evals_result_['valid_1']['binary_logloss']

        print('Fold %2d Log Loss : %.6f' % (n_fold + 1, log_loss(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full Log Loss score %.6f' % log_loss(df['target'], oof_preds))
    test['target'] = sub_preds.copy()

    # Get the average feature importance between folds
    mean_importance = importance_df.groupby('feature').mean().reset_index()
    mean_importance.sort_values(by= 'gain', ascending=False, inplace=True)

        
    # Save feature importance, test predictions and oof predictions as csv
    if GENERATE_SUBMISSION_FILES:

        # Generate oof csv
        oof = pd.DataFrame()
        oof['ID'] = df['ID'].copy()
        df['predictions'] = oof_preds.copy()
        df['target'] = df['target'].copy()
        df.to_csv('oof{}.csv'.format(SUBMISSION_SUFIX), index=False)
        
        # Save submission (test data) and feature importance
        test[['ID', 'target']].to_csv('../submission/submission{}.csv'.format(SUBMISSION_SUFIX), index=False)
        mean_importance.to_csv('feature_importance{}.csv'.format(SUBMISSION_SUFIX), index=False)
        
        plt.hist(test['target'])
        plt.show()        
        
    return mean_importance

In [ ]:
mean_importance = run_model(df, categorical_feats)

# Stacking

In [ ]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed

In [ ]:
    
stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, 
                                                          loss="huber", 
                                                          max_depth=4, 
                                                          max_features=0.55, 
                                                          min_samples_leaf=17, 
                                                          min_samples_split=14, 
                                                          subsample=0.7)),
    LassoLarsCV()
)

# Separando features preditoras e target
X = df.drop(['ID','target'], axis=1)
y = df['target']

stacked_pipeline.fit(X, y)
results = stacked_pipeline.predict(X)

#LogLoss Score on the entire Train data when averaging
#print('Log Loss score on train data:')
#print(log_loss(y, stacked_pipeline.predict(X)*0.1))
#print(log_loss(y, stacked_pipeline.predict(X)*0.2))
#print(log_loss(y, stacked_pipeline.predict(X)*0.3))
#print(log_loss(y, stacked_pipeline.predict(X)*0.4))